In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 1
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []

with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])
p_fingerprints = np.asarray(p_fingerprints)[1:]
c_fingerprints = np.asarray(c_fingerprints)[1:]
print(p_fingerprints.shape)
print(p_fingerprints[1:5,-1])

p_fingerprints = p_fingerprints.astype(int)
c_fingerprints = c_fingerprints.astype(float)
(no_examples , ip_dim) = p_fingerprints.shape
labels = labels[1:]
print(no_examples,ip_dim)
print("total no of 1s",np.sum(p_fingerprints))
print("total no of 0s",no_examples*ip_dim-np.sum(p_fingerprints))

(47831, 122)
['0' '0' '0' '0']
(47831, 122)
('total no of 1s', 256419)
('total no of 0s', 5578963)


In [4]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)

In [5]:
no_active_ele = (sum(labels2))
print(no_active_ele)

[170]


In [6]:
print labels2.shape

(47831, 1)


###  Check tsne

In [7]:
# X_embedded = TSNE(n_components=2).fit_transform(p_fingerprints)

In [8]:
# plt.figure(figsize=(8, 6)) 
# labels2 = labels2.astype(float)
# labels2 = labels2.reshape(no_examples,)
# plt.scatter(X_embedded[:, 0], X_embedded[:, 1],c = labels2)#map=plt.cm.BrOr)
# #plt.colorbar()
# plt.grid()
# plt.show()

In [9]:
batch_size = 120

z_dim = 30
comb_dim = ip_dim + z_dim

In [10]:
#### Encoder

class encoder(nn.Module):
    def __init__(self):
        super(encoder,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        #self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,z_dim)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        #x = F.leaky_relu(self.l3(x))
        x = (self.l4(x))
        
        return x

### Decoder

class decoder(nn.Module):
    def __init__(self):
        super(decoder,self).__init__()
        self.l1 = nn.Linear(z_dim,500)
        self.l2 = nn.Linear(500,500)
        #self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,ip_dim)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        #x = F.leaky_relu(self.l3(x))
        x = F.tanh(self.l4(x))
        
        return x

###  Discriminator

class disc(nn.Module):
    def __init__(self):
        super(disc,self).__init__()
        self.l1 = nn.Linear(z_dim,100)
        #self.l2 = nn.Linear(100,100)
        self.l3 = nn.Linear(100,100)
        self.l4 = nn.Linear(100,1)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.elu(self.l3(x))
        x = F.sigmoid(self.l4(x))
        
        return x

In [11]:
ind = np.arange(no_examples)
np.random.shuffle(ind)
p_fingerprints = p_fingerprints[ind]
c_fingerprints = c_fingerprints[ind]
labels2 = labels2[ind]
labels2 = labels2.astype(int)

In [12]:
def get_train_batch(batch_size,validation_iter=0,binary=True,high_pos=False):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
        #print labels_train.shape
        p_train_data = p_fingerprints
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        c_train_data = c_fingerprints[indices]
        p_train_data = p_fingerprints[indices]
        labels_train = labels2[indices]
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
     

    if binary == True:
        train_batch = p_train_data[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(int)
    else:
        train_batch = c_fingerprints[samples].reshape(batch_size,ip_dim2)
        train_batch = train_batch.astype(float)
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [13]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = p_fingerprints[s_ind:e_ind]
    else:
        train_data = c_fingerprints[s_ind:e_ind]
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    #print train_data.shape, labels_val.shape
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  
        

In [14]:
G = decoder().cuda()
E = encoder().cuda()
D = disc().cuda()

def clear_grad():
    G.zero_grad()
    E.zero_grad()
    D.zero_grad()

E_solver = optim.Adam(E.parameters(),lr = 1e-3)
G_solver = optim.Adam(G.parameters(),lr = 1e-3)
D_solver = optim.Adam(D.parameters(),lr = 1e-3)


In [15]:
def get_true_z(label):
    
    noise = np.random.randn(batch_size,z_dim)
    label = label.numpy().reshape(batch_size,1)
    #print noise.shape,label.shape
    z = 0.5*noise + label*10
    z = Variable(torch.FloatTensor(z).cuda())
    return z
    #z = torch.add(noise,value=1,other=label)
    #z = Variable(z.cuda())

In [16]:
epochs = 300
for ep in range(epochs):
    if(1==1):
    #for idx ,(x,label) in enumerate(train_loader):
        x,label = get_train_batch(batch_size,validation_iter=0)
        label = label.type(torch.FloatTensor).data
        z = get_true_z(label)
        zhat = E(x)
        xhat = G(zhat)
        
        gen_loss = nn.MSELoss()(xhat,x)
        gen_loss.backward(retain_graph=True)
        G_solver.step()
        E_solver.step()
        clear_grad()
        
        add_small = 1e-10
        d_true = D(z)
        d_false = D(zhat)
        disc_loss = -(torch.mean(torch.log(d_true + add_small) + torch.log((1-d_false) + add_small)))
        disc_loss.backward(retain_graph=True)
        D_solver.step()
        clear_grad()
        
        if(ep%3==0):
            zhat = E(x)
            d_false = D(zhat)
            encoder_loss = -torch.mean(torch.log(d_false + add_small))
            encoder_loss.backward()
            E_solver.step()
            clear_grad()
            
#     if(ep%10==0):
#         xcheck,labels = get_train_batch(batch_size=1000)
#         labels_sc = labels.cpu().data.numpy()
#         #xcheck = Variable(xcheck.view(1000,28*28).cuda())
#         zhat = E(xcheck)
#         z_mu = zhat.cpu().data.numpy()
#         plt.figure(figsize=(8, 6)) 
#         #colors = cm.rainbow(np.linspace(0, 1, 10))
#         plt.scatter(z_mu[:, 0], z_mu[:, 1],c = labels_sc,cmap=plt.cm.autumn)
#         plt.colorbar()
#         plt.grid()
#         plt.show()         
        
    print gen_loss.data[0], disc_loss.data[0], encoder_loss.data[0]

0.043869368732 1.38004851341 0.63739079237
0.0452127121389 1.3779116869 0.63739079237
0.0380492247641 1.37320196629 0.63739079237
0.0399157963693 1.35718142986 0.605000078678
0.0385810993612 1.35515403748 0.605000078678
0.0371844656765 1.35437762737 0.605000078678
0.0411791987717 1.34697973728 0.592301905155
0.0392143502831 1.35802185535 0.592301905155
0.0350101329386 1.34695827961 0.592301905155
0.0366548411548 1.34632635117 0.589863657951
0.0403345562518 1.37081861496 0.589863657951
0.0369305424392 1.35806524754 0.589863657951
0.0382278375328 1.33548855782 0.595479726791
0.0358482450247 1.36955225468 0.595479726791
0.0359015911818 1.35907411575 0.595479726791
0.0323961339891 1.29697775841 0.645729243755
0.0353337079287 1.33092427254 0.645729243755
0.0350226275623 1.28378546238 0.645729243755
0.0352188125253 1.24766612053 0.732438266277
0.0341480113566 1.24348247051 0.732438266277
0.0373438335955 1.20104777813 0.732438266277
0.035602774471 1.10204482079 0.91320925951
0.0353230386972 1

0.0317718312144 1.21018779278 0.795353710651
0.0363016761839 1.23219811916 0.795353710651
0.0354149863124 1.22510147095 0.795353710651
0.0351158715785 1.19160032272 0.841314017773
0.0332594625652 1.21786928177 0.841314017773
0.0345781445503 1.20778274536 0.841314017773
0.0351763032377 1.17294740677 0.854009568691
0.0318442173302 1.21764600277 0.854009568691
0.033702980727 1.2480006218 0.854009568691
0.033207282424 1.2129124403 0.805799543858
0.0350623205304 1.26290309429 0.805799543858
0.034061819315 1.28005802631 0.805799543858
0.0364218130708 1.2752301693 0.750113368034
0.0345775596797 1.36357009411 0.750113368034
0.0330829247832 1.35205245018 0.750113368034
0.0371064096689 1.37138366699 0.68157607317
0.0336755141616 1.44588804245 0.68157607317
0.0330670326948 1.47222709656 0.68157607317
0.0341048277915 1.4586482048 0.615107953548
0.0347646102309 1.54670405388 0.615107953548
0.0366040356457 1.54375779629 0.615107953548
0.0338670052588 1.52750885487 0.577205598354
0.0336828120053 1.61

In [17]:
xchck = Variable(torch.cuda.FloatTensor(p_fingerprints))
zchk = E(xchck)

In [18]:
# zchk = zchk.data.cpu().numpy()
# X_embedded = TSNE(n_components=2).fit_transform(zchk)

In [19]:
# plt.figure(figsize=(8, 6)) 
# plt.scatter(X_embedded[:, 0], X_embedded[:, 1],c = labels2,cmap=plt.cm.spring)
# plt.colorbar()
# plt.grid()
# plt.show()

###  Using a random forest classifier on this

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import time

In [28]:
lables2 = labels2.reshape((labels2.shape[0],))
zchk = zchk.data.cpu().numpy()

47831
[[0]
 [0]
 [0]
 [0]
 [0]]
(47831, 30)


In [44]:
tic = time.time()
clf = RandomForestClassifier(n_estimators = 30,max_depth=2, random_state=0,class_weight={0:1,1:200})
clf.fit(zchk,labels2)
toc = time.time()
print (toc - tic)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.674827098846


In [45]:
test_op = clf.predict(zchk)
cm1 = metrics.confusion_matrix(labels2,test_op)
print('tn, fp, fn, tp',cm1.ravel())

('tn, fp, fn, tp', array([43974,  3687,   129,    41]))


In [46]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints_test = []
c_fp = []
labels = []
with open(path+fname+'red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints_test.append(row[:bf])
        c_fp.append(row[bf:-1])
        labels.append(row[-1])

In [47]:
p_fingerprints_test = np.asarray(p_fingerprints_test)[1:]
p_fingerprints_test = p_fingerprints_test.astype(int)

## get z_test ##
p_test = Variable(torch.cuda.FloatTensor(p_fingerprints_test))
z_test = E(p_test)
z_test = z_test.cpu().data.numpy()
print z_test.shape

(11957, 30)


In [48]:




c_fp = np.asarray(c_fp)[1:]
c_fp = c_fp.astype(float)

c_fp = (c_fp - np.mean(c_fp,axis=0))/np.std(c_fp,axis=0)

fingerprints_test = np.concatenate((p_fingerprints_test,c_fp),axis=1)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints_test.shape
labels = labels[1:]
print(no_examples,ip_dim)
print("total no of 1s",np.sum(p_fingerprints_test))
print("total no of 0s",no_examples*ip_dim-np.sum(p_fingerprints_test))

(11957, 122)
('total no of 1s', 64311)
('total no of 0s', 1394443)


In [49]:
labels2_test = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2_test[i] = 1
    else:
        labels2_test[i] = 0
labels2_test = np.asarray(labels2_test,dtype=np.int)

In [50]:
test_op = clf.predict(z_test)
cm1 = metrics.confusion_matrix(labels2_test,test_op)
print('tn, fp, fn, tp',cm1.ravel())

('tn, fp, fn, tp', array([10961,   954,    32,    10]))
